In [44]:
import sys
import json
import csv
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from collections import Counter
from scipy.sparse import *

In [ ]:
businessjson = 'dataset/business.json'
checkinjson = 'dataset/checkin.json'

# Load Business data
business_json_lines = [json.loads( l.strip() ) for l in open(businessjson).readlines() ]
df = json_normalize(business_json_lines)



In [ ]:
print df[0]

In [ ]:
# Get records from Yelp set for only restaurants
df = df[df['categories'].astype(str).str.lower().str.contains('restaurant') == True]
df = df.reset_index(drop=True)

#Remove Columns of least significance
df = df.drop(['address','attributes.AcceptsInsurance','attributes.AgesAllowed','attributes.BYOB','attributes.ByAppointmentOnly','attributes.Corkage','attributes.DietaryRestrictions.dairy-free','attributes.DietaryRestrictions.gluten-free','attributes.DietaryRestrictions.halal','attributes.DietaryRestrictions.kosher','attributes.DietaryRestrictions.soy-free','attributes.DietaryRestrictions.vegan','attributes.DietaryRestrictions.vegetarian','attributes.HairSpecializesIn.africanamerican','attributes.HairSpecializesIn.asian','attributes.HairSpecializesIn.coloring','attributes.HairSpecializesIn.curly','attributes.HairSpecializesIn.extensions','attributes.HairSpecializesIn.kids','attributes.HairSpecializesIn.perms','attributes.HairSpecializesIn.straightperms','attributes.Open24Hours','attributes.RestaurantsCounterService','business_id','categories','hours.Friday','hours.Monday','hours.Saturday','hours.Sunday','hours.Thursday','hours.Tuesday','hours.Wednesday','latitude','longitude','name','neighborhood','state','city','postal_code'],axis=1)
df_binary = df.copy()
# Classify the records
df_binary['target'] = [1 if df_binary['stars'][x] >= 3.5 and df_binary['review_count'][x] >= 30 else 0 for x in range(len(df_binary))]
df_binary.to_csv('business_csv_filter.csv', encoding='utf-8', index=False)
print df_binary['target'].value_counts()



In [ ]:
headers = list(df.select_dtypes(include=['object']).columns.values)
#print headers
df = pd.get_dummies(data=df, columns=headers)
df = df.fillna(0)


print df.head(10)

In [ ]:
df_multi_train = df.loc[0:41300]
df_multi_train.to_csv('business_final_train.csv', encoding='utf-8', index=False)

df_multi_test = df.loc[41300:]
df_multi_test.to_csv('business_final_test.csv', encoding='utf-8', index=False)

print df_multi_train['stars'].value_counts(),"df_multi_train"
print df_multi_test['stars'].value_counts(),"df_multi_train"

In [ ]:
print "rsgrg"

In [45]:
dataset = list()
labels = list()

import csv

with open('business_multi_final_train.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    for l in your_list:
        dataset.append(l[0:-1])
        labels.append(l[-1])
print dataset[0]

['2', '0', '0', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '4']


In [ ]:
# Seggregting data to positive and negative buckets.
# Splitting train and test data for binary.
pos_bucket = df_binary.loc[df_binary['target'] == 1].reset_index(drop=True)
neg_bucket = df_binary.loc[df_binary['target'] == 0].reset_index(drop=True)

df_pos_train = df_binary.loc[0:11588]
df_pos_test = df_binary.loc[11588:]
df_neg_train = df_binary.loc[0:11884]
df_neg_test = df_binary.loc[11884:]

print df_pos_train.shape[0],"df_pos_train"
print df_pos_test.shape[0],"df_pos_test"
print df_neg_train.shape[0],"df_neg_train"
print df_neg_test.shape[0],"df_neg_test"


df_train = pd.concat([df_pos_train, df_neg_train], axis=0)
df_train = df_train.reset_index(drop=True)

print df_train.shape[0],"df_train"

df_test = pd.concat([df_pos_test, df_neg_test], axis=0)
df_test = df_test.reset_index(drop=True)

print df_test.shape[0],"df_test"
print df_train.loc[0:1]," df_train.loc_ahead"
print df_train.loc[12589]," df_train.loc_behind"

In [46]:
def build_matrix(lists):
    """ 
            Build sparse matrix from a list of documents, 
            each of which is a list of word/terms in the document.  
    """


    docs = []
    for list in lists:
        docs.append([int(n) if n != '' else 0 for n in list])
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)


    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = [k for k,_ in cnt.most_common()]
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat


In [47]:
dataset_csr = build_matrix(dataset)

In [48]:
def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [49]:
dataset_csr = csr_l2normalize(dataset_csr, copy=True)

In [74]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X_new = SelectKBest(chi2, k=30).fit_transform(dataset_csr, labels)

In [68]:
print X_new.shape

(41301, 4)


In [87]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=30)
out = clf.fit(dataset_csr, labels).predict(dataset_csr)
count = 0
for i in range(len(labels)):
    if labels[i] == out[i]:
        count = count + 1
print count
print labels[0:10]
print out[0:10]

18420
['4.5', '4.5', '2', '3', '3', '2.5', '3', '3', '2.5', '4']
['4' '4' '3.5' '4.5' '3.5' '2.5' '3' '3' '2.5' '4']


In [88]:
from sklearn.metrics import accuracy_score
print accuracy_score(labels, out)
print len(labels)

0.445994043728
41301
